In [ ]:
import pandas as pd
import numpy as np
from difflib import SequenceMatcher, get_close_matches
from tqdm import tqdm
import re

In [ ]:
df_manufacturer = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/data/unmatched_manufacturer_stockist_transactions.csv')
df_manufacturer = df_manufacturer.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)
df_manufacturer.head()

,manufacturer_name,best_manufacturer_match,manufacturer_match_score
0,shalina healthcare ltd,viva healthcare ltd.,0.79
1,pi industries ltd,zaam industries ltd.,0.79
2,kitale industries,kitale industries limited,0.79
3,avivet agro solution ltd,agrosolutions ltd.,0.79
4,local manufacturer,"""""manufacturer",0.79


In [ ]:
df_non_dup = df_manufacturer.drop_duplicates(subset='manufacturer_name', keep='first')
df_non_dup['word_count'] = df_manufacturer['manufacturer_name'].apply(lambda x: len(x.split()) if isinstance(x, str) else 1)
df_non_dup['manufacturer_name'] = df_non_dup['manufacturer_name'].astype('str')

manufacturer_list = []

for index, row in df_non_dup.iterrows():
    word_count = row['word_count']
    manufacturer_name = row['manufacturer_name']
    
    if word_count in [1,2]:
        manufacturer_slice = manufacturer_name.strip().split()[:1]
        manufacturer_list.append(' '.join(manufacturer_slice))
    elif word_count in [3,4,5]:
        manufacturer_slice = manufacturer_name.strip().split()[:2]
        manufacturer_list.append(' '.join(manufacturer_slice))
    else:
        manufacturer_slice = manufacturer_name.strip().split()[:3]
        manufacturer_list.append(' '.join(manufacturer_slice))
           
df_non_dup['manufacturer_slice'] = manufacturer_list        
df_non_dup = df_non_dup.reset_index(drop=True)
df_non_dup.head()

/tmp/ipykernel_37912/1627982089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_dup['word_count'] = df_manufacturer['manufacturer_name'].apply(lambda x: len(x.split()) if isinstance(x, str) else 1)
/tmp/ipykernel_37912/1627982089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_dup['manufacturer_name'] = df_non_dup['manufacturer_name'].astype('str')
/tmp/ipykernel_37912/1627982089.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice
0,shalina healthcare ltd,viva healthcare ltd.,0.79,3,shalina healthcare
1,pi industries ltd,zaam industries ltd.,0.79,3,pi industries
2,kitale industries,kitale industries limited,0.79,2,kitale
3,avivet agro solution ltd,agrosolutions ltd.,0.79,4,avivet agro
4,local manufacturer,"""""manufacturer",0.79,2,local


In [ ]:
df_non_dup.shape

(30392, 5)

In [ ]:
df_manufacturer_slice = df_non_dup['manufacturer_slice'].drop_duplicates(keep='first').to_frame()
df_manufacturer_slice = df_manufacturer_slice.reset_index(drop=True)
df_manufacturer_slice

,manufacturer_slice
0,shalina healthcare
1,pi industries
2,kitale
3,avivet agro
4,local
...,...
20170,0
20171,#
20172,@
20173,*


In [ ]:
# matched = {'key': ['value']}
# def compare(i):
#     compare = {}
#     for key, value in matched.items():
#       if i in value:
#         compare.update({i: value})
#       else:
#         compare.update({i: get_close_matches(i, df_manufacturer_slice['manufacturer_slice'].to_list(), 5, 0.8 )})
#     matched.update(compare.items())
#     # manufacturer_slice = list(compare.keys())
#     # match = []
#     # for key, items in compare.items():
#     #   match.append(items)
#     # return pd.Series([manufacturer_slice, match],index=['kemsa_prod', 'match'])
#     return matched
      
# for i in tqdm(range(10), desc="Processing", ncols=80, bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt}"):
#   # cleaned_manufacturers_df = pd.DataFrame()
#   # cleaned_manufacturers_df[['manufacturer_slice', 'match']] = df_manufacturer_slice['manufacturer_slice'].apply(lambda x: compare(x))
#   # cleaned_manufacturers_df = cleaned_manufacturers_df.applymap(lambda x: x[0] if x else '')
#   # cleaned_manufacturers_df.head()
#   matched_df = df_manufacturer_slice['manufacturer_slice'].apply(lambda x: compare(x))

# # x = compare('pioneer  feeds')
# # cleaned_manufacturers_df = pd.DataFrame()
# # cleaned_manufacturers_df[['manufacturer_slice', 'match']] = x
# # cleaned_manufacturers_df['match'][0]
# x

In [ ]:
matched = []
def compare(i):
    compare = {}
    if i in matched:
        compare.update({i: ''})
    else:
        compare.update({i: get_close_matches(i, df_manufacturer_slice['manufacturer_slice'].to_list(), 20, 0.8)})
    # matched.extend(compare.values())
    matched.extend([item for sublist in compare.values() for item in sublist])
    manufacturer_slice = list(compare.keys())
    match = []
    for key, items in compare.items():
      match.append(items)
    return pd.Series([manufacturer_slice, match],index=['manufacturer_slice', 'match'])

      
cleaned_manufacturers_df = pd.DataFrame()
cleaned_manufacturers_df[['manufacturer_slice', 'match']] = df_manufacturer_slice['manufacturer_slice'][:10].apply(lambda x: compare(x))
cleaned_manufacturers_df = cleaned_manufacturers_df.applymap(lambda x: x[0] if x else '')
cleaned_manufacturers_df.head()

,manufacturer_slice,match
0,shalina healthcare,"[shalina healthcare, shelina healthcare, shali..."
1,pi industries,"[pi industries, pi industies, pt industries, p..."
2,kitale,"[kitale, kutale, kitali, kita]"
3,avivet agro,"[avivet agro, avvivet agro, avivet ago, avivet..."
4,local,"[local, lockal, localy, locals, localp, locall..."


In [10]:
matched

[['shalina healthcare',
  'shelina healthcare',
  'shalina health',
  'skant healthcare',
  'sakar healthcare',
  'sakari healthcare',
  'pharma healthcare'],
 ['pi industries',
  'pi industies',
  'pt industries',
  'p1 industries',
  'ip industries',
  'pl lndustries',
  'primier industries',
  'premier industries',
  'parijat industries',
  'y.s.p industries',
  'sespo industries',
  'sepso industries',
  's.a.p industries',
  'light industries',
  'grain industries',
  'sun industries',
  'ram industries',
  'bhd industries',
  'pidilite industries',
  'sukari industries'],
 ['kitale', 'kutale', 'kitali', 'kita'],
 ['avivet agro',
  'avvivet agro',
  'avivet ago',
  'avivet. agro.',
  'vet agro',
  'veta agro'],
 ['local',
  'lockal',
  'localy',
  'locals',
  'localp',
  'locall',
  'local.',
  'local,',
  'llocal',
  'locl',
  'loca',
  'loal',
  'lcal',
  'locally',
  'locallo',
  'rocal',
  'pocal',
  'loyal',
  'loxal',
  'locsl'],
 ['sigma', 'sigima', 'sanigma', 'simba', 'sai

In [11]:
dnd_cleaned_manufacturers_df = cleaned_manufacturers_df.copy()

In [15]:
# cleaned_manufacturers_df[cleaned_manufacturers_df['match'] == ['']]
filtered_df = cleaned_manufacturers_df[cleaned_manufacturers_df['match'].apply(lambda x: x == '')]
filtered_df

,manufacturer_slice,match


In [82]:
cleaned_manufacturers_df = pd.read_csv('/home/natasha/Documents/Iprocure/Clustering-for-Product-Matching/model/cleaned_manufacturers.csv')
# cleaned_manufacturers_df['match'] = cleaned_manufacturers_df['match'].apply(lambda x: x[:])
cleaned_manufacturers_df

,manufacturer_slice,match
0,shalina healthcare ltd,"['shalina healthcare ltd', 'shalina healthcare..."
1,pi industries ltd,"['pi industries ltd', 'pi industies ltd', 'pt ..."
2,kitale industries,"['kitale industries', 'kitale indusries', 'kit..."
3,avivet agro,"['avivet agro', 'avivets agro', 'avivet agro ..."
4,local manufacturer,"['local manufacturer', 'local manufacturers', ..."
...,...,...
29074,0,['0']
29075,#,['#']
29076,@,['@']
29077,*,['*']


In [83]:
cleaned_manufacturers_df['manufacturer_slice'] = cleaned_manufacturers_df['manufacturer_slice'].astype('str')

In [89]:
cleaned_manufacturers_df_subset = cleaned_manufacturers_df[:20]

for i, row in cleaned_manufacturers_df_subset[:20].iterrows():
    string = row['manufacturer_slice']
    lst = row['match']
    
    # Check if the string exists in any list of previous rows
    for prev_i in range(i):
        prev_lst = cleaned_manufacturers_df_subset.at[prev_i, 'match']
        if string in prev_lst:
            cleaned_manufacturers_df_subset.at[i, 'match'] = prev_lst
            break  # Stop searching after finding the first match

# Reset the index of the DataFrame
cleaned_manufacturers_df_subset = cleaned_manufacturers_df_subset.reset_index(drop=True)
pd.set_option('display.max_colwidth', None)
cleaned_manufacturers_df_subset

,manufacturer_slice,match
0,shalina healthcare ltd,"['shalina healthcare ltd', 'shalina healthcare', 'shalina healtcare', 'skant healthcare ltd', 'sakar healthcare ltd']"
1,pi industries ltd,"['pi industries ltd', 'pi industies ltd', 'pt industries ltd', 'p1 industries ltd', 'ip industries ltd']"
2,kitale industries,"['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries']"
3,avivet agro,"['avivet agro', 'avivets agro', 'avivet agro sol', 'avivet agro ltd', 'vet agro']"
4,local manufacturer,"['local manufacturer', 'local manufacturers', 'local manufaturer', 'local manufacture', 'excel manufacturers']"
5,sigma limited,"['sigma limited', 'sidai limited', 'daima limited', 'gal limited', 'sigma ltd']"
6,bestvet agrosolution ltd,"['bestvet agrosolution ltd', 'avivet agrosolutions l.t.d']"
7,crown chemicals,"['crown chemicals', 'crownchemicals', 'canon chemicals', 'abcon chemicals', 'dow chemicals']"
8,pl lndustries ltd,"['pl lndustries ltd', 'pt industries ltd', 'pi industries ltd', 'p1 industries ltd', 'ip industries ltd']"
9,africasia seeds,"['africasia seeds', 'africasia seed', 'aficasia seeds', 'afrcassia seeds', 'aficasia seed']"


In [81]:
cleaned_manufacturers_df[:20]

,manufacturer_slice,match
0,shalina healthcare ltd,"['shalina healthcare ltd', 'shalina healthcare..."
1,pi industries ltd,"['pi industries ltd', 'pi industies ltd', 'pt ..."
2,kitale industries,"['kitale industries', 'kitale indusries', 'kit..."
3,avivet agro,"['avivet agro', 'avivets agro', 'avivet agro ..."
4,local manufacturer,"['local manufacturer', 'local manufacturers', ..."
5,sigma limited,"['sigma limited', 'sidai limited', 'daima limi..."
6,bestvet agrosolution ltd,"['bestvet agrosolution ltd', 'avivet agrosolut..."
7,crown chemicals,"['crown chemicals', 'crownchemicals', 'canon c..."
8,pl lndustries ltd,"['pl lndustries ltd', 'pt industries ltd', 'pi..."
9,africasia seeds,"['africasia seeds', 'africasia seed', 'aficasi..."


In [72]:
dnd = df_non_dup.copy()

In [73]:
df_non_dup = pd.merge(df_non_dup, cleaned_manufacturers_df, how='left', on='manufacturer_slice')
df_non_dup

,manufacturer_name,best_manufacturer_match,manufacturer_match_score,word_count,manufacturer_slice,match
0,shalina healthcare ltd,viva healthcare ltd.,0.79,3,shalina healthcare ltd,"['shalina healthcare ltd', 'shalina healthcare..."
1,pi industries ltd,zaam industries ltd.,0.79,3,pi industries ltd,"['pi industries ltd', 'pi industies ltd', 'pt ..."
2,kitale industries,kitale industries limited,0.79,2,kitale industries,"['kitale industries', 'kitale indusries', 'kit..."
3,avivet agro solution ltd,agrosolutions ltd.,0.79,4,avivet agro,"['avivet agro', 'avivets agro', 'avivet agro ..."
4,local manufacturer,"""""manufacturer",0.79,2,local manufacturer,"['local manufacturer', 'local manufacturers', ..."
...,...,...,...,...,...,...
30388,0,"virbrac,1ere avenue,2065 m-lid,06510,carros fr...",0.04,1,0,"['0ak medical', 'oak medicals', 'oak medical'..."
30389,#,"globion india pvt ltd, india #811 10th na main...",0.02,1,#,"['anicare enterprise', 'armcare enterprises', ..."
30390,@,"zzagro vietnam co.,ltd-vietnam",0.00,1,@,"['afivet', 'afrivet', 'afrivets', 'afri_vet', ..."
30391,*,"zzagro vietnam co.,ltd-vietnam",0.00,1,*,"['jun*long', 'jinlong']"


In [77]:
import pandas as pd

# Assuming you have a DataFrame called 'df' with columns 'col1' and 'col2'

# Initialize an empty list to store the similar strings
similar_strings = []

# Iterate over the rows of the DataFrame
for i, row in df_non_dup.iterrows():
    string = row['manufacturer_name']
    value = row['match']
    
    # Check if any other row has a similar value in 'col2'
    similar_rows = df_non_dup[df_non_dup['match'] == value]
    
    # Extract the strings from 'col1' in similar rows
    similar_strings.append(similar_rows['manufacturer_name'].tolist())

# Remove duplicates from the list of similar strings
# similar_strings = list(set(similar_strings))

len(similar_strings)

30393

In [71]:
cleaned_manufacturers_df[cleaned_manufacturers_df['manufacturer_slice'].isin(['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries'])]['match'].to_list()

["['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries']",
 "['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries']",
 "['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries']",
 "['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries']",
 "['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries']"]

In [70]:
cleaned_manufacturers_df['match'][2]

"['kitale industries', 'kitale indusries', 'kitale industrues', 'kitale industriez', 'kitale induatries']"

In [ ]:
# matches = []
# for index, row in cleaned_manufacturers_df.iterrows():
#     matches.extend(row['match'])

In [50]:
# cleaned_manufacturers_df.to_csv('cleaned_manufacturers.csv', index = False)

In [57]:
# dictionary = cleaned_manufacturers_df[:5].to_dict()
# dictionary

# for key, value in dictionary:
#     print(key, value)

### Topic Modeling

In [ ]:
# extracting most common words from each cluster in order
cluster_word_freq = {}

for doc, cluster_label in zip(unique_product_names, labels):
    words = re.split(r'\s+|-|\(|\)|/|\\|\||,', doc)
    for word in words:
        if cluster_label in cluster_word_freq:
            cluster_word_freq[cluster_label][word] = cluster_word_freq[cluster_label].get(word, 0) + 1
        else:
            cluster_word_freq[cluster_label] = {word: 1}
    
for cluster_label in cluster_word_freq:
    cluster_word_freq[cluster_label] = sorted(cluster_word_freq[cluster_label].items(), key=lambda x: x[1], reverse=True)
    
cluster_word_freq_df = pd.DataFrame.from_dict(cluster_word_freq.items())
cluster_word_freq_df.rename(columns={0: 'cluster_id', 1: 'word_freq'}, inplace=True)
cluster_word_freq_df.head()